In [1]:
import tensorflow as tf
from bert import run_classifier, modeling, optimization, tokenization
from datetime import datetime
import os
import nbimporter
from config import get_train_config, get_test_15s

Importing Jupyter notebook from config.ipynb


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# model parametes
MAX_SEQ_LENGTH = 128
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 10.0
WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 50
NUM_TRAIN_SIZE = 4462

In [3]:
# get config based on specified language
OUTPUT_CKPT_DIR, OUTPUT_PB_DIR, VOCAB_FILE, BERT_CONFIG_FILE, INIT_CHECKPOINT, SHUFFLE_TFR, TRAIN_TFR, TEST_TFR, VALIDATE_TFR, LABELS_LIST = get_train_config(
    "cn")

In [4]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(config=bert_config,
                               is_training=is_training,
                               input_ids=input_ids,
                               input_mask=input_mask,
                               token_type_ids=segment_ids,
                               use_one_hot_embeddings=use_one_hot_embeddings)
    output_layer = model.get_pooled_output()
    hidden_size = output_layer.shape[-1].value
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels],
                                  initializer=tf.zeros_initializer())
    with tf.variable_scope("loss"):
        if is_training:
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        probabilities = tf.nn.softmax(logits, axis=-1)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        predicted_labels = tf.squeeze(
            tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, per_example_loss, logits, probabilities,
                predicted_labels)

def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):
        """The `model_fn` for TPUEstimator."""

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" %
                            (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
            is_real_example = tf.cast(features["is_real_example"],
                                      dtype=tf.float32)
        else:
            is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities,
         predicted_labels) = create_model(bert_config, is_training, input_ids,
                                          input_mask, segment_ids, label_ids,
                                          num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(
                 tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint,
                                                  assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                            init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            tf.logging.info("**** Do Train ****")
            train_op = optimization.create_optimizer(total_loss, learning_rate,
                                                     num_train_steps,
                                                     num_warmup_steps, use_tpu)

            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold_fn=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:
            tf.logging.info("**** Do Validate ****")

            def metric_fn(per_example_loss, label_ids, logits,
                          is_real_example):
                predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                accuracy = tf.metrics.accuracy(labels=label_ids,
                                               predictions=predictions,
                                               weights=is_real_example)
                loss = tf.metrics.mean(values=per_example_loss,
                                       weights=is_real_example)
                return {
                    "eval_accuracy": accuracy,
                    "eval_loss": loss,
                }

            eval_metrics = (metric_fn, [
                per_example_loss, label_ids, logits, is_real_example
            ])
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metrics=eval_metrics,
                scaffold_fn=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.PREDICT:
            tf.logging.info("**** Do Predict ****")
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                predictions={
                    "probabilities": probabilities,
                    "predicted_labels": predicted_labels
                },
                scaffold_fn=scaffold_fn)
        else:
            raise ValueError(
                "Only Train, Validate and Predict modes are supported: %s" %
                (mode))
        return output_spec

    return model_fn

In [5]:
def bulid_estimator(init_checkpoint):
    tf.logging.set_verbosity(tf.logging.INFO)
    bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG_FILE)
    if MAX_SEQ_LENGTH > bert_config.max_position_embeddings:
        raise ValueError(
            "Cannot use sequence length %d because the BERT model was only trained up to sequence length %d"
            % (MAX_SEQ_LENGTH, bert_config.max_position_embeddings))
    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
    run_config = tf.contrib.tpu.RunConfig(
        cluster=None,
        master=None,
        model_dir=OUTPUT_CKPT_DIR,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=SAVE_SUMMARY_STEPS,
            num_shards=8,
            per_host_input_for_training=is_per_host))
    num_train_step = int(NUM_TRAIN_SIZE / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_step * WARMUP_PROPORTION)

    model_fn = model_fn_builder(bert_config=bert_config,
                                num_labels=len(LABELS_LIST),
                                init_checkpoint=init_checkpoint,
                                learning_rate=LEARNING_RATE,
                                num_train_steps=num_train_step,
                                num_warmup_steps=num_warmup_steps,
                                use_tpu=False,
                                use_one_hot_embeddings=False)

    estimator = tf.contrib.tpu.TPUEstimator(use_tpu=False,
                                            model_fn=model_fn,
                                            config=run_config,
                                            train_batch_size=BATCH_SIZE,
                                            eval_batch_size=8,
                                            predict_batch_size=8)
    return estimator, num_train_step

In [8]:
def do_train():
    estimator, num_train_step = bulid_estimator(INIT_CHECKPOINT)
    train_input_fn = run_classifier.file_based_input_fn_builder(
        input_file=SHUFFLE_TFR,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=True)
    print("=============== begin to train ===============")
    current_time = datetime.now()
    estimator.train(input_fn=train_input_fn, max_steps=num_train_step)
    print("=============== end of train ===============")
    print("Training took time ", datetime.now() - current_time)

In [9]:
do_train()

INFO:tensorflow:Using config: {'_protocol': None, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7c798c2cf8>, '_save_summary_steps': 100, '_log_step_count_steps': None, '_global_id_in_cluster': 0, '_task_id': 0, '_save_checkpoints_steps': 500, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=50, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_eval_distribute': None, '_master': '', '_tf_random_seed': None, '_num_worker_replicas': 1, '_cluster': None, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_device_fn': None, '_is_chief': True, '_evaluation_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_secs': None, '_model_dir': './out

INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflo

INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKP

INFO:tensorflow:examples/sec: 66.183
INFO:tensorflow:global_step/sec: 2.07879
INFO:tensorflow:examples/sec: 66.5212
INFO:tensorflow:Saving checkpoints for 1000 into ./output/chinese/checkpoint/model.ckpt.
INFO:tensorflow:global_step/sec: 1.97914
INFO:tensorflow:examples/sec: 63.3325
INFO:tensorflow:global_step/sec: 2.08526
INFO:tensorflow:examples/sec: 66.7283
INFO:tensorflow:global_step/sec: 2.08627
INFO:tensorflow:examples/sec: 66.7607
INFO:tensorflow:global_step/sec: 2.08674
INFO:tensorflow:examples/sec: 66.7755
INFO:tensorflow:Saving checkpoints for 1394 into ./output/chinese/checkpoint/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0009814446.
INFO:tensorflow:training_loop marked as finished
=============== end of train ===============
Training took time  0:12:07.841005


In [33]:
def do_test():
    estimator, _ = bulid_estimator(init_checkpoint=OUTPUT_CKPT_DIR)
    predict_input_fn = run_classifier.file_based_input_fn_builder(
        input_file=SHUFFLE_TFR,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)
    result = estimator.predict(input_fn=predict_input_fn)
    return result

In [45]:
result = do_test()

INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_train_distribute': None, '_evaluation_master': '', '_device_fn': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_task_type': 'worker', '_save_checkpoints_secs': None, '_save_checkpoints_steps': 500, '_log_step_count_steps': None, '_model_dir': './output/chinese/checkpoint', '_is_chief': True, '_num_ps_replicas': 0, '_protocol': None, '_tpu_config': TPUConfig(iterations_per_loop=50, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_eval_distribute': None, '_tf_random_seed': None, '_cluster': None, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5edec9b710>, '_task_id': 0, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, 

In [46]:
def get_accuracy_from_result(model_results, labels_list, shuffle_csv):
    ori_labels = []
    pred_labels = []
    wrong_list = []
    with open(shuffle_csv, "r") as f:
        data = f.readlines()
        data = data[1:]
        for sens in data:
            ori_labels.append(sens.split(",")[0])
    for re in model_results:
        pred_labels.append(labels_list[re["probabilities"].argmax(axis=-1)])
    rep_num = 0
    for i in range(len(ori_labels)):
        if ori_labels[i] == pred_labels[i]:
            rep_num += 1
        else:
            wrong_list.append((i, ori_labels[i], pred_labels[i]))
    accuracy = rep_num / len(ori_labels)
    return accuracy, wrong_list

In [47]:
accuracy, wrong_list = get_accuracy_from_result(
    result, LABELS_LIST, "./data/car_reviews_final/chinese/shuffle.csv")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (21128, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorfl

In [48]:
accuracy

0.9996414485478666

In [50]:
wrong_list

[(148, '外观', '空间'), (296, '内饰', '外观')]

In [6]:
def sentences_list_to_input_fn(sentences):
    tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE,
                                           do_lower_case=True)
    input_examples = [
        run_classifier.InputExample(guid="test-" + str(i),
                                    text_a=tokenization.convert_to_unicode(x),
                                    text_b=None,
                                    label=LABELS_LIST[0])
        for (i, x) in enumerate(sentences)
    ]
    input_features = run_classifier.convert_examples_to_features(
        examples=input_examples,
        label_list=LABELS_LIST,
        max_seq_length=MAX_SEQ_LENGTH,
        tokenizer=tokenizer)
    input_fn = run_classifier.input_fn_builder(features=input_features,
                                               seq_length=MAX_SEQ_LENGTH,
                                               is_training=False,
                                               drop_remainder=False)
    return input_fn

In [7]:
def get_prediction_for_sentences(estimator, sentences):
    input_fn = sentences_list_to_input_fn(sentences)
    predictions = estimator.predict(input_fn)
    return [(sentence, prediction['probabilities'],
             LABELS_LIST[prediction['predicted_labels']])
            for sentence, prediction in zip(sentences, predictions)]

In [6]:
estimator, _ = bulid_estimator(init_checkpoint=OUTPUT_CKPT_DIR)

INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_log_step_count_steps': None, '_protocol': None, '_save_checkpoints_secs': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': 500, '_global_id_in_cluster': 0, '_model_dir': './output/chinese/checkpoint', '_num_worker_replicas': 1, '_task_id': 0, '_evaluation_master': '', '_device_fn': None, '_tpu_config': TPUConfig(iterations_per_loop=50, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5f235c3828>, '_service': None, '_cluster': None, '_eval_distribute': None, '_task_type': 'worker', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_train_distribute': None, '_experimental_distribute': None, '_save_summary_steps': 100, '_tf_ra

In [ ]:
sentences,sentences_labels = get_test_15s("cn")

In [12]:
predictions = get_prediction_for_sentences(estimator, sentences)

INFO:tensorflow:Writing example 0 of 15
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] 还 没 护 磨 合 期 ， 现 在 1800 公 里 ， 油 耗 9 . 5l ， 比 新 车 已 经 降 了 不 少 ， ， 还 有 下 降 的 趋 势 [SEP]
INFO:tensorflow:input_ids: 101 6820 3766 2844 4836 1394 3309 8024 4385 1762 8961 1062 7027 8024 3779 5450 130 119 9892 8024 3683 3173 6756 2347 5307 7360 749 679 2208 8024 8024 6820 3300 678 7360 4638 6633 1232 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:  name = bert/encoder/layer_0/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorfl

INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = be

INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/kernel:0, shape = (3072, 768), *INI

In [13]:
predictions_labels = [(p_label[2],o_label) for p_label,o_label in zip(predictions,sentences_labels)]

In [14]:
predictions_labels

[('能耗', '能耗'),
 ('能耗', '能耗'),
 ('外观', '外观'),
 ('空间', '空间'),
 ('动力', '动力'),
 ('动力', '动力'),
 ('能耗', '能耗'),
 ('内饰', '内饰'),
 ('外观', '外观'),
 ('安全辅助', '安全辅助'),
 ('能耗', '能耗'),
 ('操控', '操控'),
 ('操控', '操控'),
 ('能耗', '能耗'),
 ('动力', '动力')]

In [8]:
def savd_model_with_estimator(model_dir, estimator):
    def serving_input_fn_single():
        label_ids = tf.placeholder(tf.int32, [None], name='label_ids')
        input_ids = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH],
                                   name='input_ids')
        input_mask = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH],
                                    name='input_mask')
        segment_ids = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH],
                                     name='segment_ids')
        input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
            'label_ids':
            label_ids,
            'input_ids':
            input_ids,
            'input_mask':
            input_mask,
            'segment_ids':
            segment_ids,
        })()
        return input_fn

    def serving_input_fn_batch():
        feature_columns = []
        feature_keys = ['label_ids', 'input_ids', 'input_mask', 'segment_ids']
        for key in feature_keys:
            if key == 'label_ids':
                feature_columns.append(
                    tf.feature_column.numeric_column(key=key,
                                                     shape=[],
                                                     dtype=tf.int64))
            else:
                feature_columns.append(
                    tf.feature_column.numeric_column(key=key,
                                                     shape=[MAX_SEQ_LENGTH],
                                                     dtype=tf.int64))
        feature_spec = tf.feature_column.make_parse_example_spec(
            feature_columns=feature_columns)
        serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
            feature_spec)
        return serving_input_receiver_fn

    estimator._export_to_tpu = False
    estimator.export_savedmodel(model_dir, serving_input_fn_batch())

In [12]:
OUTPUT_CKPT_DIR

'./output/chinese/checkpoint'

In [13]:
estimator, _ = bulid_estimator(init_checkpoint=OUTPUT_CKPT_DIR)

INFO:tensorflow:Using config: {'_is_chief': True, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_device_fn': None, '_num_ps_replicas': 0, '_protocol': None, '_num_worker_replicas': 1, '_cluster': None, '_tf_random_seed': None, '_save_checkpoints_secs': None, '_experimental_distribute': None, '_evaluation_master': '', '_save_summary_steps': 100, '_service': None, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe29a577f60>, '_model_dir': './output/chinese/checkpoint', '_eval_distribute': None, '_save_checkpoints_steps': 500, '_tpu_config': TPUConfig(iterations_per_loop=50, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_global_id_in_cluster': 0, '_keep_checkpoint_every_n_hours': 10000, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_trai

In [14]:
savd_model_with_estimator(model_dir=OUTPUT_PB_DIR,estimator=estimator)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (21128, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/l

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorfl